# Feature engineering and selection

In [1]:
import pandas as pd
import numpy as np

from collections import defaultdict

In [2]:
def load_data_set():
    train_data = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    test["outcome"] = 9
    bids = pd.read_csv('bids.csv')
    bids = bids.sort(['auction', 'time'])
    return train_data,bids,test

train,bids,test = load_data_set()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [53]:
'''
Alors fonction generer pour les bids l'ensemble des donnees suivantes : 
Pour chaque vente : temps moyen,max, std et min entre deux bids d une meme personne
                    temps moyen,max, st entre deux bids
                    nombre de bids max, min, et moyen de chaque utilisateur 
sauvegarde des resultats dans 3 csv differents
'''
def time_into_auction(train,bids,test):
    human = train[train.outcome==0].bidder_id
    bot = train[train.outcome==1].bidder_id
    grouped = bids.groupby('auction')
    done = -5
    #temps de bid par rapport aux dans une meme vente
    times_df = {}
    #temps de bid par rapport a soit meme pour chaque vente
    bid_himselft = {}
    #number of bids:
    number_bids = {}
    remaining = len(grouped)
    for bides in grouped:
        sub_data= bides[1]
        difference=  [0,0]
        last_bids = {}
        current_bids = {}
        for acc in sub_data.iterrows():
            i,line = acc
            cle = line['bidder_id']
            if difference[0] == 0:
                difference[0] = line['time']
            else:
                if not cle in times_df:
                    times_df[cle] = []
                times_df[cle].append(line['time']-difference[0])
                difference[0] = line['time']
            if cle not in last_bids:
                last_bids[cle] = (line['time'],[])
            else:
                last_bids[cle][1].append(line['time']-last_bids[cle][0])
                last_bids[cle] = (line['time'],last_bids[cle][1])
            if cle not in current_bids:
                current_bids[cle] = 1
            else:
                current_bids[cle] +=1
        #fusion des dictionnaires
        for c in current_bids:
            if not c in number_bids:
                number_bids[c] = [current_bids[c]]
            else:
                number_bids[c].append(current_bids[c])
        for c in last_bids:
            if not c in bid_himselft and len(last_bids[c][1])>0:
                bid_himselft[c] = last_bids[c]
            elif  len(last_bids[c][1])>0:
                bid_himselft[c] += last_bids[c]
        if remaining %200 ==0:
            print("remaining : ", remaining)
        remaining -=1
    df = []
    for cle in times_df.keys():
        df.append({'bidder_id': cle, 'mean_autres': np.mean(times_df[cle]),
    'min_autres': np.min(times_df[cle]), 'max_autres': np.max(times_df[cle]),'std_autres': np.std(times_df[cle])})
    pd.DataFrame(df).to_csv('res/time_auction.csv', index=False)
    df = []
    for cle in number_bids.keys():
        df.append({'bidder_id': cle, 'mean_number_bids': np.mean(number_bids[cle]),
    'min_number_bids': np.min(number_bids[cle]), 'max_number_bids': np.max(number_bids[cle]),
    'number_bids': len(number_bids[cle]),'std_number_bids': np.std(number_bids[cle])})
    pd.DataFrame(df).to_csv('res/bids_number.csv', index=False)
    df  = []
    for cle in bid_himselft.keys():
        val = bid_himselft[cle][1]
        df.append({'bidder_id': cle, 'mean_time_bids': np.mean(val),
    'min_time bids': np.min(val), 'max_time_bids': np.max(val),
    'number_of_auctions_bids': len(val),'std_time_bids': np.std(val)})
    pd.DataFrame(df).to_csv('res/himslef_time.csv', index=False)

recreate_feature = False
if recreate_feature:
    time_into_auction(train,bids,test)

'''How to detect a bot ?
Same ip
Fast biding
Bid faster than human
#
average time between two bids
time since bid starting
number of bids
max time

faire par rapport aux autres utilisateurs et faire par rapport a lui meme
average number of bids per auction
max number of bids per auction
'''

'How to detect a bot ?\nSame ip\nFast biding\nBid faster than human\n#\naverage time between two bids\ntime since bid starting\nnumber of bids\nmax time\n\nfaire par rapport aux autres utilisateurs et faire par rapport a lui meme\naverage number of bids per auction\nmax number of bids per auction\n'

In [ ]:
# generate the features using the features created before
def feature_creation():
    print("ok")